<a href="https://colab.research.google.com/github/Dvaalmeyda/deteksi-kemiripan-judul-skripsi/blob/main/Gen_AI_TEST_Skripsi_Diva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skripsi

- Diva Putra Almeyda
- 2010501053
- S1 Teknik Elektro Universitas Tidar

## Evaluasi Kinerja Parafrase Tools Generative AI - Semantic Similarity Scoring

### Dependencies

In [ ]:
import pandas as pd
import re
import nltk
import torch
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel

In [ ]:
# Download NLTK data
nltk.download('punkt_tab')
nltk.download('stopwords')

# Load stopwords Bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load Dataset

In [ ]:
# URL dataset
gpt_label1_dataset_url = 'https://docs.google.com/spreadsheets/d/1bDpX0Xf02X3AlSyWfV5JveURUxkqcjmBQC13fTWRDMw/export?gid=1011515501&format=csv'
gpt_label0_dataset_url = 'https://docs.google.com/spreadsheets/d/1bDpX0Xf02X3AlSyWfV5JveURUxkqcjmBQC13fTWRDMw/export?gid=1811247420&format=csv'
claude_label1_dataset_url = 'https://docs.google.com/spreadsheets/d/1bDpX0Xf02X3AlSyWfV5JveURUxkqcjmBQC13fTWRDMw/export?gid=1818322094&format=csv'
claude_label0_dataset_url = 'https://docs.google.com/spreadsheets/d/1bDpX0Xf02X3AlSyWfV5JveURUxkqcjmBQC13fTWRDMw/export?gid=1451007939&format=csv'

In [ ]:
gpt1_df = pd.read_csv(gpt_label1_dataset_url)
gpt0_df = pd.read_csv(gpt_label0_dataset_url)
claude1_df = pd.read_csv(claude_label1_dataset_url)
claude0_df = pd.read_csv(claude_label0_dataset_url)

### Preprocess

#### Filtering

In [ ]:
# Filtering & Casefolding
def remove_punctuation(text):
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'\s+', ' ', text)
  text = text.lower()
  return text

#### Tokenize nltk & Stopwords removal

In [ ]:
# Helper Function Tokenisasi nltk
def tokenize_nltk_stopwords(text):
    # Tokenisasi
    tokens = word_tokenize(text)
    # Stopwords removal
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

# Fungsi untuk menggabungkan token kembali menjadi kalimat
def join_tokens(tokens):
    return ' '.join(tokens)

#### Main Preprocess

In [ ]:
# Main Function - Preprocessing
def preprocess_text(text):
    text = remove_punctuation(text)
    tokens = tokenize_nltk_stopwords(text)
    processed_text = join_tokens(tokens)
    return processed_text

### Hitung Similarity

In [ ]:
# Load pre-trained model tokenisasi Indobert
indobert_tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
# Load model
model = AutoModel.from_pretrained('indolem/indobert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

#### Embedding

In [ ]:
# Helper Function - embedding text
def get_embedding(text, max_length=50):
    # Tokenisasi dan dapatkan embeddings dari model
    inputs = indobert_tokenizer(text,
                                return_tensors="pt",
                                max_length=max_length,
                                truncation=True,
                                padding='max_length')

    with torch.no_grad():
        outputs = model(**inputs)
    # Ambil per token [CLS]
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return cls_embedding

#### Semantic Similarity Scoring

In [ ]:
# Helper Function untuk menghitung similarity
def calculate_similarity(text1, text2):
    embed1 = get_embedding(text1)
    embed2 = get_embedding(text2)
    # Hitung cosine similarity
    similarity = cosine_similarity(embed1.reshape(1, -1), embed2.reshape(1, -1))
    return similarity[0][0]

### Main Program

#### Preprocess Dataset

In [ ]:
# Preprocess data gpt label 1
gpt1_df['judul 1 processed'] = gpt1_df['judul 1'].apply(preprocess_text)
gpt1_df['judul 2 processed'] = gpt1_df['judul 2'].apply(preprocess_text)
# Preprocess data gpt label 0
gpt0_df['judul 1 processed'] = gpt0_df['judul 1'].apply(preprocess_text)
gpt0_df['judul 2 processed'] = gpt0_df['judul 2'].apply(preprocess_text)
# Preprocess data claude label 1
claude1_df['judul 1 processed'] = claude1_df['judul 1'].apply(preprocess_text)
claude1_df['judul 2 processed'] = claude1_df['judul 2'].apply(preprocess_text)
# Preprocess data claude label 0
claude0_df['judul 1 processed'] = claude0_df['judul 1'].apply(preprocess_text)
claude0_df['judul 2 processed'] = claude0_df['judul 2'].apply(preprocess_text)

#### Scoring Dataset

In [ ]:
# Menghitung similarity untuk dataset gpt label 1
gpt1_df['nilai kemiripan semantik'] = gpt1_df.apply(
    lambda row: calculate_similarity(row['judul 1'], row['judul 2']), axis=1
)

# Semantic Similarity gpt label 0
gpt0_df['nilai kemiripan semantik'] = gpt0_df.apply(
    lambda row: calculate_similarity(row['judul 1'], row['judul 2']), axis=1
)

# Semantic Similarity Claude label 1
claude1_df['nilai kemiripan semantik'] = claude1_df.apply(
    lambda row: calculate_similarity(row['judul 1'], row['judul 2']), axis=1
)

# Semantic Similarity Claude label 0
claude0_df['nilai kemiripan semantik'] = claude0_df.apply(
    lambda row: calculate_similarity(row['judul 1'], row['judul 2']), axis=1
)

### Labeling

In [ ]:
# Labeling threshold 0.5
threshold = 0.5

# Label gpt label 1
gpt1_df['label'] = gpt1_df['nilai kemiripan semantik'].apply(lambda x: 1 if x >= threshold else 0)

# Label gpt label 0
gpt0_df['label'] = gpt0_df['nilai kemiripan semantik'].apply(lambda x: 1 if x >= threshold else 0)

# Label claude label 1
claude1_df['label'] = claude1_df['nilai kemiripan semantik'].apply(lambda x: 1 if x >= threshold else 0)

# Label claude label 0
claude0_df['label'] = claude0_df['nilai kemiripan semantik'].apply(lambda x: 1 if x >= threshold else 0)

### Result

In [ ]:
gpt1_df

,id,judul 1,judul 2,judul 1 processed,judul 2 processed,nilai kemiripan semantik,label
0,1,Analisis Gangguan Tidak Seimbang pada Jaring D...,Studi ketidakseimbangan pada jaringan distribu...,analisis gangguan seimbang jaring distribusi 2...,studi ketidakseimbangan jaringan distribusi 20...,0.863577,1
1,5,Analisis Pengaruh Pembebanan Terhadap Posisi P...,Pengaruh beban terhadap perubahan posisi tap c...,analisis pengaruh pembebanan posisi peubah sad...,pengaruh beban perubahan posisi tap changer tr...,0.849191,1
2,9,Analisis Pengurangan Harmonik Pada Converter A...,Kajian penurunan harmonik pada konverter AC-DC...,analisis pengurangan harmonik converter acdc t...,kajian penurunan harmonik konverter acdc filte...,0.627730,1
3,11,Analisis Redaman Lintasan Sinyal Di Luar Bangu...,Evaluasi redaman sinyal di area luar (outdoor)...,analisis redaman lintasan sinyal bangunan outd...,evaluasi redaman sinyal area outdoor sistem ko...,0.727167,1
4,17,Implementasi Ic Tl 494 Pada Catu Daya Arus Sea...,Penerapan IC TL 494 untuk sumber daya DC 5 Volt,implementasi ic tl 494 catu daya arus searah 5...,penerapan ic tl 494 sumber daya dc 5 volt,0.795733,1
5,25,Implementasi Regulator Lm723 Dan Lm317 Pada Ca...,Penggunaan regulator LM723 dan LM317 dalam pen...,implementasi regulator lm723 lm317 catu daya s...,penggunaan regulator lm723 lm317 pengaturan te...,0.802632,1
6,29,Relai Differensial Sebagai Proteksi Utama Tran...,Pemanfaatan relai diferensial untuk perlindung...,relai differensial proteksi utama transformato...,pemanfaatan relai diferensial perlindungan tra...,0.669425,1
7,31,Aktivasi Dan Instalasi Pada Antena Microwave D...,Instalasi dan pengaktifan antena microwave di ...,aktivasi instalasi antena microwave pt trimba ...,instalasi pengaktifan antena microwave pt trim...,0.907326,1
8,33,Disappear Automatically Retaliatory Target (Da...,Target Otomatis Menghilang dan Membalas (DART)...,disappear automatically retaliatory target dar...,target otomatis menghilang membalas dart divis...,0.760809,1
9,53,Studi Penyaluran Dan Pusat Pengatur Beban Jawa...,Penelitian distribusi dan pusat pengendalian b...,studi penyaluran pusat pengatur beban jawa bal...,penelitian distribusi pusat pengendalian beban...,0.687751,1


In [ ]:
gpt0_df

,id,judul 1,judul 2,judul 1 processed,judul 2 processed,nilai kemiripan semantik,label
0,1,Analisis Gangguan Tidak Seimbang pada Jaring D...,Studi kestabilan sistem seimbang pada jaringan...,analisis gangguan seimbang jaring distribusi 2...,studi kestabilan sistem seimbang jaringan tran...,0.347526,0
1,5,Analisis Pengaruh Pembebanan Terhadap Posisi P...,Studi optimalisasi pengurangan beban pada posi...,analisis pengaruh pembebanan posisi peubah sad...,studi optimalisasi pengurangan beban posisi sa...,0.810058,1
2,9,Analisis Pengurangan Harmonik Pada Converter A...,Studi peningkatan harmonik pada inverter DC-AC...,analisis pengurangan harmonik converter acdc t...,studi peningkatan harmonik inverter dcac tapis...,0.327637,0
3,11,Analisis Redaman Lintasan Sinyal Di Luar Bangu...,Studi penguatan lintasan sinyal dalam bangunan...,analisis redaman lintasan sinyal bangunan outd...,studi penguatan lintasan sinyal bangunan indoo...,0.667552,1
4,17,Implementasi Ic Tl 494 Pada Catu Daya Arus Sea...,Penerapan IC LM393 pada pengubah daya arus bol...,implementasi ic tl 494 catu daya arus searah 5...,penerapan ic lm393 pengubah daya arus bolakbal...,0.876932,1
5,25,Implementasi Regulator Lm723 Dan Lm317 Pada Ca...,Pengembangan regulator switching berbasis IC T...,implementasi regulator lm723 lm317 catu daya s...,pengembangan regulator switching berbasis ic t...,0.612736,1
6,29,Relai Differensial Sebagai Proteksi Utama Tran...,Relai arus lebih sebagai proteksi cadangan tra...,relai differensial proteksi utama transformato...,relai arus proteksi cadangan transformator dis...,0.790757,1
7,31,Aktivasi Dan Instalasi Pada Antena Microwave D...,Pengujian dan kalibrasi antena satelit di PT. ...,aktivasi instalasi antena microwave pt trimba ...,pengujian kalibrasi antena satelit pt satnet e...,0.317044,0
8,33,Disappear Automatically Retaliatory Target (Da...,Automatic Real-time Target Identifier (ARTI) T...,disappear automatically retaliatory target dar...,automatic realtime target identifier arti tran...,0.219901,0
9,53,Studi Penyaluran Dan Pusat Pengatur Beban Jawa...,Analisis distribusi dan monitoring pembangkit ...,studi penyaluran pusat pengatur beban jawa bal...,analisis distribusi monitoring pembangkit jawa...,0.633281,1


In [ ]:
claude1_df

,id,judul 1,judul 2,judul 1 processed,judul 2 processed,nilai kemiripan semantik,label
0,1,Analisis Gangguan Tidak Seimbang pada Jaring D...,Investigasi Ketidakselarasan Distribusi Listri...,analisis gangguan seimbang jaring distribusi 2...,investigasi ketidakselarasan distribusi listri...,0.366475,0
1,5,Analisis Pengaruh Pembebanan Terhadap Posisi P...,Studi Pengaruh Beban Terhadap Mekanisme Pengal...,analisis pengaruh pembebanan posisi peubah sad...,studi pengaruh beban mekanisme pengalihan tap ...,0.772725,1
2,9,Analisis Pengurangan Harmonik Pada Converter A...,Evaluasi Reduksi Harmonik dalam Konverter AC-D...,analisis pengurangan harmonik converter acdc t...,evaluasi reduksi harmonik konverter acdc imple...,0.634454,1
3,11,Analisis Redaman Lintasan Sinyal Di Luar Bangu...,Kajian Peredaman Transmisi Sinyal Eksternal pa...,analisis redaman lintasan sinyal bangunan outd...,kajian peredaman transmisi sinyal eksternal si...,0.675752,1
4,17,Implementasi Ic Tl 494 Pada Catu Daya Arus Sea...,Aplikasi Integrated Circuit TL 494 dalam Desai...,implementasi ic tl 494 catu daya arus searah 5...,aplikasi integrated circuit tl 494 desain sumb...,0.849016,1
5,25,Implementasi Regulator Lm723 Dan Lm317 Pada Ca...,Pengembangan Regulator Tegangan LM723 dan LM31...,implementasi regulator lm723 lm317 catu daya s...,pengembangan regulator tegangan lm723 lm317 si...,0.723734,1
6,29,Relai Differensial Sebagai Proteksi Utama Tran...,Analisis Relai Diferensial untuk Proteksi Prim...,relai differensial proteksi utama transformato...,analisis relai diferensial proteksi primer tra...,0.670967,1
7,31,Aktivasi Dan Instalasi Pada Antena Microwave D...,Implementasi dan Konfigurasi Antena Microwave ...,aktivasi instalasi antena microwave pt trimba ...,implementasi konfigurasi antena microwave pt t...,0.257749,0
8,33,Disappear Automatically Retaliatory Target (Da...,Analisis Sistem Respons Otomatis DART pada Kom...,disappear automatically retaliatory target dar...,analisis sistem respons otomatis dart komunika...,0.627061,1
9,53,Studi Penyaluran Dan Pusat Pengatur Beban Jawa...,Kajian Distribusi dan Manajemen Beban Wilayah ...,studi penyaluran pusat pengatur beban jawa bal...,kajian distribusi manajemen beban wilayah jawa...,0.619403,1


In [ ]:
claude0_df

,id,judul 1,judul 2,judul 1 processed,judul 2 processed,nilai kemiripan semantik,label
0,1,Analisis Gangguan Tidak Seimbang pada Jaring D...,Sintesis Keseimbangan pada Jaringan Transmisi ...,analisis gangguan seimbang jaring distribusi 2...,sintesis keseimbangan jaringan transmisi 20kv ...,0.378489,0
1,5,Analisis Pengaruh Pembebanan Terhadap Posisi P...,Evaluasi Pengurangan Beban Terhadap Stabilitas...,analisis pengaruh pembebanan posisi peubah sad...,evaluasi pengurangan beban stabilitas transfor...,0.610202,1
2,9,Analisis Pengurangan Harmonik Pada Converter A...,Peningkatan Harmonik pada Konverter DC-AC deng...,analisis pengurangan harmonik converter acdc t...,peningkatan harmonik konverter dcac metode akt...,0.756977,1
3,11,Analisis Redaman Lintasan Sinyal Di Luar Bangu...,Amplifikasi Sinyal Dalam Bangunan (Indoor) pad...,analisis redaman lintasan sinyal bangunan outd...,amplifikasi sinyal bangunan indoor sistem komu...,0.559513,1
4,17,Implementasi Ic Tl 494 Pada Catu Daya Arus Sea...,Dekonstruksi Fungsi IC Alternatif pada Sumber ...,implementasi ic tl 494 catu daya arus searah 5...,dekonstruksi fungsi ic alternatif sumber daya ...,0.351126,0
5,25,Implementasi Regulator Lm723 Dan Lm317 Pada Ca...,Modifikasi Regulator Non-Standar untuk Sumber ...,implementasi regulator lm723 lm317 catu daya s...,modifikasi regulator nonstandar sumber daya te...,0.245364,0
6,29,Relai Differensial Sebagai Proteksi Utama Tran...,Analisis Kegagalan Sistem Proteksi Sekunder Tr...,relai differensial proteksi utama transformato...,analisis kegagalan sistem proteksi sekunder tr...,0.609234,1
7,31,Aktivasi Dan Instalasi Pada Antena Microwave D...,Dekonstruksi dan Deinstaasi Sistem Komunikasi ...,aktivasi instalasi antena microwave pt trimba ...,dekonstruksi deinstaasi sistem komunikasi kabe...,0.239193,0
8,33,Disappear Automatically Retaliatory Target (Da...,Konstruksi Persistensi Strategis pada Sistem K...,disappear automatically retaliatory target dar...,konstruksi persistensi strategis sistem komuni...,0.565481,1
9,53,Studi Penyaluran Dan Pusat Pengatur Beban Jawa...,Evaluasi Pembatasan Distribusi Beban di Wilaya...,studi penyaluran pusat pengatur beban jawa bal...,evaluasi pembatasan distribusi beban wilayah a...,0.543017,1


### Evaluasi

In [ ]:
# Hasil ChatGPT
label1_gpt = (gpt1_df['label']==1).sum()
label0_gpt = (gpt0_df['label']==0).sum()

print('Hasil dengan bantuan ChatGPT\n')
print('Total Data Terlabel 1:', label1_gpt)
print('Total Data Terlabel 0:', label0_gpt)
print('Akurasi:', (label1_gpt + label0_gpt) / (len(gpt1_df) + len(gpt0_df)))

Hasil dengan bantuan ChatGPT

Total Data Terlabel 1: 14
Total Data Terlabel 0: 12
Akurasi: 0.65


In [ ]:
# Hasil Claude.ai
label1_claude = (claude1_df['label']==1).sum()
label0_claude = (claude0_df['label']==0).sum()

print('Hasil dengan bantuan Chatclaude\n')
print('Total Data Terlabel 1:', label1_claude)
print('Total Data Terlabel 0:', label0_claude)
print('Akurasi:', (label1_claude + label0_claude) / (len(claude1_df) + len(claude0_df)))

Hasil dengan bantuan Chatclaude

Total Data Terlabel 1: 11
Total Data Terlabel 0: 13
Akurasi: 0.6


### Export to XLSX

In [ ]:
# Tambahkan kolom keterangan
gpt1_df['keterangan'] = 'gpt label 1'
gpt0_df['keterangan'] = 'gpt label 0'
claude1_df['keterangan'] = 'claude label 1'
claude0_df['keterangan'] = 'claude label 0'

In [ ]:
combined_df = pd.concat([gpt1_df, gpt0_df, claude1_df, claude0_df], ignore_index=True)
combined_df.to_excel('GEN AI PARAPHRASE MEASUREMENT.xlsx', index=False)